In [1]:
import boto3

In [2]:
s3 = boto3.client("s3")

In [3]:
response = s3.list_objects_v2(Bucket ="llm-finetune-datset-bucket",Prefix ="datasets/")

In [4]:
response

{'ResponseMetadata': {'RequestId': 'KEKF3V6QPESXHQ2F',
  'HostId': 'pXUGYXlcZDSu2KygzYq0ubiWfKw5G5X/GCmb88d1YiBYOBAcRjhnyz4Ok0+GS+7ryPmuW3v30ks+q3WXygzWBw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pXUGYXlcZDSu2KygzYq0ubiWfKw5G5X/GCmb88d1YiBYOBAcRjhnyz4Ok0+GS+7ryPmuW3v30ks+q3WXygzWBw==',
   'x-amz-request-id': 'KEKF3V6QPESXHQ2F',
   'date': 'Sat, 21 Feb 2026 09:04:36 GMT',
   'x-amz-bucket-region': 'us-east-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'datasets/',
   'LastModified': datetime.datetime(2026, 2, 20, 8, 42, 23, tzinfo=tzlocal()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'Key': 'datasets/pharma_instruction_data.csv',
   'LastModified': datetime.datetime(2026, 2, 20, 10, 39, 34, tzinfo=tzlocal()),
   'ETa

In [5]:
for obj in response.get("Contents",[]):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2026, 2, 20, 8, 42, 23, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2026, 2, 20, 10, 39, 34, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [6]:
for obj in response.get("Contents",[]):
    print(obj["Key"])

datasets/
datasets/pharma_instruction_data.csv


In [7]:
dataset_path = "s3://llm-finetune-datset-bucket/datasets/pharma_instruction_data.csv"

In [8]:
from datasets import load_dataset

In [9]:
dataset_path

's3://llm-finetune-datset-bucket/datasets/pharma_instruction_data.csv'

In [10]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={"train": "pharma_instruction_data.csv"}
)

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5
    })
})

In [12]:
print(dataset["train"])

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})


In [13]:
# import subprocess
# import sys

# packages = [
#     "ipykernel",
#     "transformers==4.44.2",
#     "datasets==2.21.0",
#     "sagemaker",
#     "boto3",
#     "pandas",
#     "cmake==3.30.2",
#     "pyarrow==17.0.0",
#     "bitsandbytes==0.43.3",
#     "peft==0.12.0",
#     "s3fs==2024.9.0",
#     "accelerate==0.34.0",
#     "tqdm",
#     "torch",
# ]

# for pkg in packages:
#     print(f"Installing {pkg}...")
#     result = subprocess.run(
#         [sys.executable, "-m", "pip", "install", pkg],
#         capture_output=True, text=True
#     )
#     if result.returncode == 0:
#         print(f"✅ {pkg} installed successfully")
#     else:
#         print(f"❌ {pkg} failed: {result.stderr[-200:]}")

Installing ipykernel...
✅ ipykernel installed successfully
Installing transformers==4.44.2...
✅ transformers==4.44.2 installed successfully
Installing datasets==2.21.0...
✅ datasets==2.21.0 installed successfully
Installing sagemaker...
✅ sagemaker installed successfully
Installing boto3...
✅ boto3 installed successfully
Installing pandas...
✅ pandas installed successfully
Installing cmake==3.30.2...
✅ cmake==3.30.2 installed successfully
Installing pyarrow==17.0.0...
✅ pyarrow==17.0.0 installed successfully
Installing bitsandbytes==0.43.3...
✅ bitsandbytes==0.43.3 installed successfully
Installing peft==0.12.0...
✅ peft==0.12.0 installed successfully
Installing s3fs==2024.9.0...
✅ s3fs==2024.9.0 installed successfully
Installing accelerate==0.34.0...
✅ accelerate==0.34.0 installed successfully
Installing tqdm...


KeyboardInterrupt: 

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5
    })
})

In [15]:
print(dataset['train'][0])

{'instruction': 'Explain the mechanism of action of Metformin.', 'input': None, 'output': 'Metformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'}


In [16]:
def format_example(example):
    prompt = f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    
    return {"text": prompt}

In [17]:
dataset = dataset.map(format_example)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 5
    })
})

In [19]:
dataset['train']['text'][0]

'### Instruction:\nExplain the mechanism of action of Metformin.\n\n### Input:\nNone\n\n### Response:\nMetformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'

In [20]:
dataset['train']['text'][1]

'### Instruction:\nList two non-glycemic benefits of Metformin mentioned in the text.\n\n### Input:\nNone\n\n### Response:\nImproved cardiovascular outcomes; anti-inflammatory effects. (Also suggested: anticancer potential via mTOR inhibition.)'

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [22]:
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [24]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [25]:
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [26]:
tokenized = dataset.map(tokenize_fn, batched=True)

Parameter 'function'=<function tokenize_fn at 0x7fbe545a8900> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [27]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
})

In [28]:
from peft import LoraConfig, get_peft_model, TaskType

In [29]:
model = AutoModelForCausalLM.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

RuntimeError: unable to mmap 4400216536 bytes from file </home/ec2-user/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/59f6f375b26bde864a6ca194a9a3044570490064/model.safetensors>: Cannot allocate memory (12)

In [30]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [31]:
model = get_peft_model(model, lora_config)

NameError: name 'model' is not defined